### LOAD DATASET

In [ ]:
import numpy as np
import json
from PIL import Image, ImageDraw

def create_mask_from_annotation(annot, img_size):
    mask = Image.new('L', img_size, 0)
    for shape in annot['shapes']:
        polygon = [(float(x), float(y)) for x, y in shape['points']]
        ImageDraw.Draw(mask).polygon(polygon, outline=1, fill=1)
    return np.array(mask)

def load_image_and_mask(img_path, annot_path, img_size=(128, 128)):
    img = Image.open(img_path)
    img = img.resize(img_size)
    img = np.array(img, dtype=np.float32) / 255.0  # Normalize and cast to float32

    with open(annot_path, 'r') as file:
        annot = json.load(file)
    mask = create_mask_from_annotation(annot, img_size)
    mask = np.array(mask, dtype=np.uint8)  # Cast mask to uint8
    mask = mask.reshape((*mask.shape, 1))  # Add channel dimension

    return img, mask



def load_dataset(image_dir, annotation_dir, batch_size=16):
    image_paths = sorted([str(path) for path in Path(image_dir).glob("*.jpg")])
    annotation_paths = sorted([str(path) for path in Path(annotation_dir).glob("*.json")])

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, annotation_paths))
    dataset = dataset.map(lambda x, y: tf.numpy_function(
                            load_image_and_mask, 
                            [x, y], 
                            [tf.float32, tf.uint8]),  # Specify output types here
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (tf.reshape(x, [128, 128, 3]), tf.reshape(y, [128, 128, 1])))
    dataset = dataset.batch(batch_size)
    return dataset
train_dataset = load_dataset('path/to/data/image', 'path/to/data/annotations')
val_dataset = load_dataset('path/to/data/image', 'path/to/data/annotations')
test_dataset = load_dataset('path/to/data/image', )

### LOADING AND RUNNING MODEL

In [ ]:
import HRnet as HRNet
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

HRNet.model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])  # Replace 'accuracy' with your IoU metrics

# Training parameters
total_epochs = 90
report_interval = 30

# Lists to store metrics
metrics_history = []

# Training and Evaluation Loop
for epoch in range(total_epochs):
    print(f"Starting epoch {epoch + 1}/{total_epochs}")

    # Training step
    for x_batch_train, y_batch_train in train_dataset:
        with tf.GradientTape() as tape:
            logits = HRNet.model(x_batch_train, training=True)
            loss_value = tf.keras.losses.sparse_categorical_crossentropy(y_batch_train, logits)
        grads = tape.gradient(loss_value, HRNet.model.trainable_weights)
        HRNet.model.optimizer.apply_gradients(zip(grads, HRNet.model.trainable_weights))

    # Validation and Test step at specified intervals
    if (epoch + 1) % report_interval == 0 or epoch == total_epochs - 1:
        val_miou, test_miou = [], []
        # Validation Evaluation
        for x_batch_val, y_batch_val in val_dataset:
            val_iou_list = HRNet.IoU_per_class(y_batch_val, HRNet.model(x_batch_val, training=False)) # Replace with actual IoU calculation
            val_miou_value = HRNet.compute_mIoU(y_batch_val, HRNet.model(x_batch_val, training=False)) # Replace with actual mIoU calculation
            val_miou.append(val_miou_value)

        # Test Evaluation
        for x_batch_test, y_batch_test in test_dataset:
            test_iou_list = HRNet.IoU_per_class(y_batch_test, HRNet.model(x_batch_test, training=False)) # Replace with actual IoU calculation
            test_miou_value = HRNet.compute_mIoU(y_batch_test, HRNet.model(x_batch_test, training=False)) # Replace with actual mIoU calculation
            test_miou.append(test_miou_value)

        avg_val_miou = np.mean(val_miou)
        avg_test_miou = np.mean(test_miou)

        metrics_history.append({
            'Epoch': epoch + 1,
            'Validation mIoU': avg_val_miou,
            'Test mIoU': avg_test_miou
        })
        print(f'Epoch {epoch + 1} - Validation mIoU: {avg_val_miou}, Test mIoU: {avg_test_miou}')

# Convert to DataFrame and save to CSV
results_df = pd.DataFrame(metrics_history)
results_df.to_csv('training_results.csv', index=False)

# Visualization
epochs = results_df['Epoch']
val_miou = results_df['Validation mIoU']
test_miou = results_df['Test mIoU']

plt.figure(figsize=(8, 4))
plt.plot(epochs, val_miou, label='Validation mIoU', color='blue')
plt.plot(epochs, test_miou, label='Test mIoU', color='green')
plt.xlabel('Epoch')
plt.ylabel('Mean IoU')
plt.title('Mean IoU over Epochs')
plt.legend()
plt.show()
